In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Building a demand forecasting model using BigQuery ML

In this notebook we show how a time series model can be trained, analysed and deployed using BigQuery ML. It provides an end-to-end solution for forecasting multiple products. Using the public dataset [Iowa Liquor Sales](https://console.cloud.google.com/marketplace/details/obfuscated-ga360-data/obfuscated-ga360-data?filter=solution-type:dataset), we build five time series models with a few SQL queries, each model predicting the retail sales of a single liquor product. 

By the end of this notebook, we will have learnt how to:
* Prepare time series data into the correct format required to build the model.
* Train a time series model in BigQuery ML.
* Evaluate the model.
* Make predictions about future demand using the model.

## Setup

### Required Libraries

In [ ]:
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import pandas as pd
from google.cloud import bigquery

### Plotting-Script (for later use)

In [ ]:
def plot_historical_and_forecast(input_timeseries, 
                                 timestamp_col_name, 
                                 data_col_name, 
                                 forecast_output=None, 
                                 actual=None, 
                                 title=None,
                                 plotstartdate=None):

    if plotstartdate:
        input_timeseries[timestamp_col_name] = pd.to_datetime(input_timeseries[timestamp_col_name])
        input_timeseries = input_timeseries[input_timeseries[timestamp_col_name] >= pd.to_datetime(plotstartdate)]
        
    input_timeseries = input_timeseries.sort_values(timestamp_col_name)    
    
    # Plot the input historical data
    plt.figure(figsize=(20,6))
    plt.plot(input_timeseries[timestamp_col_name], input_timeseries[data_col_name], label = 'Historical')
    plt.xlabel(timestamp_col_name)
    plt.ylabel(data_col_name)

    if forecast_output is not None:
        forecast_output = forecast_output.sort_values('forecast_timestamp')
        forecast_output['forecast_timestamp'] = pd.to_datetime(forecast_output['forecast_timestamp'])
        x_data = forecast_output['forecast_timestamp']
        y_data = forecast_output['forecast_value']
        confidence_level = forecast_output['confidence_level'].iloc[0] * 100
        low_CI = forecast_output['confidence_interval_lower_bound']
        upper_CI = forecast_output['confidence_interval_upper_bound']
        # Plot the forecast data
        plt.plot(x_data, y_data, alpha = 1, label = 'Forecast', linestyle='--')
        # Shade the confidence interval
        plt.fill_between(x_data, low_CI, upper_CI, color = '#539caf', alpha = 0.4, 
                         label = f'{confidence_level} confidence interval')

    # Plot actual data
    if actual is not None:
        actual = actual.sort_values(timestamp_col_name)
        plt.plot(actual[timestamp_col_name], actual[data_col_name], label = 'Actual', linestyle='--')   

    # Display title, legend
    plt.title(f'{title}', fontsize= 16)
    plt.legend(loc = 'upper center', prop={'size': 16})

### Create a BigQuery dataset

We need to specify `US` as location to be able to copy data from the public dataset into our dataset, which is also located in US.

In [ ]:
%%bigquery
CREATE SCHEMA IF NOT EXISTS
bqmlforecast
OPTIONS (
    location="US"
)

## Prepare the training data

We train the time series models on a dataset of transaction data. Each row represents a transaction for a single product, identified by the value `item_description`, and contains details such as the number of bottles sold and the sales amount in dollars. In the following steps, we use the value for the number of bottles sold to forecast product demand.

In [ ]:
%%bigquery

SELECT 
    invoice_and_item_number,
    date,
    store_number,
    item_description,
    bottles_sold,
    sale_dollars
FROM
  `bigquery-public-data.iowa_liquor_sales.sales` 
LIMIT 
  5

### Set start and end date of the train data

We can adjust the parameters `TRAININGDATA_STARTDATE` and `TRAININGDATA_ENDDATE` to specify the start/end date of the training data:

In [ ]:
ARIMA_PARAMS = {
    'TRAININGDATA_STARTDATE': '2020-01-01',
    'TRAININGDATA_ENDDATE': '2022-01-01',
}
ARIMA_PARAMS

### Write train data into a table

Some days in the data have no transactions for a specific product. BigQueryML can automatically perform some typical pre-processing:

* Missing values: values are imputed using local linear interpolation.
* Duplicated timestamps: The values are averaged over duplicated timestamps.
* Spike and dip anomalies: The values are standardised using local z-scores.

In [ ]:
%%bigquery --params $ARIMA_PARAMS

CREATE OR REPLACE TABLE bqmlforecast.training_data AS (
    WITH topsellingitems AS(
         SELECT 
            item_description,
            count(item_description) cnt_transactions
        FROM
            `bigquery-public-data.iowa_liquor_sales.sales` 
        GROUP BY 
            item_description
        ORDER BY cnt_transactions DESC
        LIMIT 5
    )
    SELECT 
        date,
        item_description AS item_name,
        SUM(bottles_sold) AS total_amount_sold
    FROM
        `bigquery-public-data.iowa_liquor_sales.sales` 
    GROUP BY
        date, item_name
    HAVING 
        date BETWEEN @TRAININGDATA_STARTDATE AND @TRAININGDATA_ENDDATE
        AND item_description IN (SELECT item_description FROM topsellingitems)
    );

SELECT 
    date,
    item_name,
    total_amount_sold
FROM 
    bqmlforecast.training_data 
ORDER BY item_name, date
LIMIT 10

### Sales history of the target spirits

We store the training data in the Pandas dataframe `pdfhistorical`:

In [ ]:
%%bigquery dfhistorical

SELECT * FROM bqmlforecast.training_data

In [ ]:
itemslist = list(dfhistorical.item_name.unique())

for item in itemslist:
    
    datah = dfhistorical[dfhistorical.item_name==item]
    plot_historical_and_forecast(input_timeseries = datah, 
                                 timestamp_col_name = "date",
                                 data_col_name = "total_amount_sold", 
                                 forecast_output = None, 
                                 actual = None,
                                 title = item)

## Model training

As the model is trained for multiple products in a single modelling statement, we specify the `item_name` column for the [TIME_SERIES_ID_COL](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-time-series#time_series_id_col) parameter. We do not need this information for a single target item.
Further information on the SQL statement for creating models: [Documentation on creating BigQuery ML time series models](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-time-series#create_model_syntax).

A common problem with time series data are holiday effects and seasonalities. BigQueryML can take this into account by specifying the `HOLIDAY_REGION`. By default, the modelling of holiday effects is deactivated. If holiday effects are activated, outliers that occur during public holidays are no longer treated as anomalies. Since we are analysing data from Iowa here, we set the `HOLIDAY_REGION` to `US`. However, there is also holiday data for other countries.

In [ ]:
%%bigquery

CREATE OR REPLACE MODEL bqmlforecast.arima_model

OPTIONS(
  MODEL_TYPE='ARIMA',
  TIME_SERIES_TIMESTAMP_COL='date', 
  TIME_SERIES_DATA_COL='total_amount_sold',
  TIME_SERIES_ID_COL='item_name',
  HOLIDAY_REGION='US'
) AS

SELECT 
    date,
    item_name,
    total_amount_sold
FROM
  bqmlforecast.training_data

### Model evaluation

We can use [ML.EVALUATE](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate) to display the quality metrics of all created models. 

The columns `non_seasonal_`{`p`,`d`,`q`} and `has_drift` define the time series model. The columns `log_likelihood`, `AIC` and `variance` are relevant for model fitting. In model fitting, the best model is determined using the [auto.ARIMA algorithm](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-time-series#auto_arima) for each time series individually.

In [ ]:
%%bigquery
SELECT * FROM ML.EVALUATE(MODEL bqmlforecast.arima_model)

We can see that five models have been trained, one for each product. Each model has its own hyperparameters, and the recognised seasonality for these five models is `WEEKLY` and partly `YEARLY`.

## Predictions

We can use [ML.FORECAST](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-forecast) to make predictions that predict the next _n_ values as specified in the `horizon` parameter. We can also optionally change the `confidence_level` to change the percentage of future values that fall within the prediction interval. The forecast data is stored in the DataFrame `dfforecast` so that it can be displayed in a later step.

In [ ]:
%%bigquery dfforecast

DECLARE HORIZON STRING DEFAULT "30";
DECLARE CONFIDENCE_LEVEL STRING DEFAULT "0.90";

EXECUTE IMMEDIATE format("""
    SELECT
      *
    FROM 
      ML.FORECAST(MODEL bqmlforecast.arima_model, 
                  STRUCT(%s AS horizon, 
                         %s AS confidence_level)
                 )
    """,HORIZON,CONFIDENCE_LEVEL)

In [ ]:
dfforecast.head()

Since `horizon` is set to 30, the result is 30 x (number of products), with one row per predicted value:

In [ ]:
print(f"Number of rows: {dfforecast.shape[0]}")

#### Visualizing predictions

We can append the predictions to the time series and thus get a visual impression of whether the predictions are plausible:

In [ ]:
itemslist = list(dfhistorical.item_name.unique())

for item in itemslist:
    datah = dfhistorical[dfhistorical.item_name==item].copy()
    dataf = dfforecast[dfforecast.item_name==item].copy()
    
    plot_historical_and_forecast(input_timeseries = datah, 
                                 timestamp_col_name = "date", 
                                 data_col_name = "total_amount_sold",
                                 forecast_output = dataf, 
                                 actual = None,
                                 title = item,
                                 plotstartdate = "2021-01-01")

#### Compare to the actual sales

We first extract the actual sales from the comparative period

In [ ]:
%%bigquery dfactual --params $ARIMA_PARAMS

DECLARE HORIZON STRING DEFAULT "30";

SELECT 
    date,
    item_description AS item_name,
    SUM(bottles_sold) AS total_amount_sold
FROM
    `bigquery-public-data.iowa_liquor_sales.sales` 
GROUP BY
    date, item_name
HAVING 
    date BETWEEN DATE_ADD(@TRAININGDATA_ENDDATE, 
                              INTERVAL 1 DAY) 
            AND DATE_ADD(@TRAININGDATA_ENDDATE, 
                             INTERVAL 1+CAST(HORIZON AS INT64) DAY) 
ORDER BY
    date;

We insert the actual sales into the visualisation above.

In [ ]:
itemslist = list(dfhistorical.item_name.unique())

for item in itemslist:
    datah = dfhistorical[dfhistorical.item_name==item].sort_values('date')
    dataf = dfforecast[dfforecast.item_name==item].sort_values(['forecast_timestamp'])
    dataa = dfactual[dfactual.item_name==item].sort_values('date')
    plot_historical_and_forecast(input_timeseries = datah, 
                             timestamp_col_name = "date", 
                             data_col_name = "total_amount_sold", 
                             forecast_output = dataf, 
                             actual = dataa,
                             title = item,
                             plotstartdate = "2021-06-01")